# Encrypted Deep Learning in PyTorch

In [136]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [137]:
# Toy Dataset
data = torch.tensor([[0,1], [0,1], [1,0], [1,1.]], requires_grad=True)
targets = torch.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [138]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

model = Model()

In [139]:
optimizer = optim.SGD(model.parameters(), lr=0.05)
for e in range(10):
    optimizer.zero_grad()
    
    output = model(data)
    loss = ((output - targets) ** 2).sum()
    loss.backward()
    optimizer.step()
    
    print(loss.data)

tensor(2.7528)
tensor(0.7895)
tensor(0.4549)
tensor(0.3149)
tensor(0.2242)
tensor(0.1644)
tensor(0.1253)
tensor(0.1017)
tensor(0.0868)
tensor(0.0768)


In [140]:
preds = model(data)

In [141]:
preds

tensor([[0.0730],
        [0.0730],
        [1.1048],
        [0.7815]], grad_fn=<AddmmBackward>)

### Using Encryption

In [142]:
import syft as sy

In [143]:
hook = sy.TorchHook(torch)

W0630 19:19:10.989034 4638279104 hook.py:97] Torch was already hooked... skipping hooking process


In [144]:
bob = sy.VirtualWorker(hook, id='bob').add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id='alice').add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id='sec_worker').add_worker(sy.local_worker)

W0630 19:19:11.206663 4638279104 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0630 19:19:11.208549 4638279104 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0630 19:19:11.210836 4638279104 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [145]:
encrypted_model = model.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [146]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:285826575 -> bob:13012554282]
 	-> (Wrapper)>[PointerTensor | me:39682574782 -> alice:48140103045]
 	*crypto provider: sec_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:20206922371 -> bob:72159487081]
 	-> (Wrapper)>[PointerTensor | me:16360797443 -> alice:10638635001]
 	*crypto provider: sec_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:60564368515 -> bob:23879604697]
 	-> (Wrapper)>[PointerTensor | me:54933155226 -> alice:67422394187]
 	*crypto provider: sec_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:88837381256 -> bob:36074538654]
 	-> (Wrapper)>[PointerTensor | me:47105925163 -> alice:48642872749]
 	*cry

In [147]:
encrypted_data = data.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [148]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:36607840833 -> bob:60693690409]
	-> (Wrapper)>[PointerTensor | me:77198462042 -> alice:55980195832]
	*crypto provider: sec_worker*

In [149]:
encrypted_prediction = encrypted_model(encrypted_data)

In [150]:
decrypted_prediction = encrypted_prediction.get().float_precision()

In [151]:
decrypted_prediction

tensor([[0.0720],
        [0.0720],
        [1.1020],
        [0.7790]])